In [ ]:
import json
import pandas as pd
from pandas.io.json import json_normalize
import time
import os
import csv
import Levenshtein as lev

In [ ]:
#set directory
outputDir = 'scraped_json//'

#combine 247 Files
two47Files = ['247_bigten.json', '247_bigtwelve.json', '247_sec.json']
main247 = []

for file in two47Files:
    conference = json.loads(open(outputDir + file, "r", encoding="utf-8").read())
    for player in conference:
        main247.append(player)

#combine rivals
rivalsFiles = ['rivals_bigten.json', 'rivals_bigtwelve.json', 'rivals_sec.json']
mainRivals = []

for file in rivalsFiles:
    conference = json.loads(open(outputDir + file, "r", encoding="utf-8").read())
    for player in conference:
        mainRivals.append(player)

#combine cfb ref
cfbRefFiles = ['cfb_ref_bigten.json', 'cfb_ref_bigtwelve.json', 'cfb_ref_sec.json']
mainCfbRef = []

for file in cfbRefFiles:
    conference = json.loads(open(outputDir + file, "r", encoding="utf-8").read())
    for player in conference:
        mainCfbRef.append(player)
                     

#Get NFL
nfl_draft_data = json.load(open(outputDir + 'nfldraft.json'))

#All American
aaData = json.load(open(outputDir + 'aaSelections.json'))

#combine All Conference
allConfFiles = ['SECAllConf.json', 'Ten12AllConf.json']
mainAllConf = []

for file in allConfFiles:
    conference = json.loads(open(outputDir + file, "r", encoding="utf-8").read())
    for player in conference:
        mainAllConf.append(player)

#Test Collections
two47_test = [{'school': 'maryland', 'year': '2002', '247_player': 'Randy Earle', '247_hs': 'Farmingdale Senior', '247_city': 'Farmingdale', '247_state': '1', '247_ranking': '0.9000', '247_pos': '9', '247_height': '75.0', '247_weight': '225', '247_stars': '4', '247_natl': '140'},{'school': 'maryland', 'year': '2002', '247_player': 'Andrew Thornton', '247_hs': 'Farmingdale Senior', '247_city': 'Farmingdale', '247_state': '1', '247_ranking': '0.9000', '247_pos': '9', '247_height': '75.0', '247_weight': '225', '247_stars': '4', '247_natl': '140'}]
nfl_test = [{'year': '2005', 'draft_round': '1', 'draft_pick': '12', 'team': 'SDG', 'player': 'Shawne Merriman', 'pos': 'LB', 'all_pros_first_team': '1', 'pro_bowls': '3', 'years_as_primary_starter': '4', 'g': '75', 'college_id': 'Maryland'}, {'year': '2005', 'draft_round': '1', 'draft_pick': '12', 'team': 'SDG', 'player': 'Joseph Holm', 'pos': 'LB', 'all_pros_first_team': '1', 'pro_bowls': '3', 'years_as_primary_starter': '4', 'g': '75', 'college_id': 'Maryland'}]
#rivals_test= [{'school': 'maryland', 'year': '2002', 'rivals_player': 'Jo Jo Walker', 'rivals_city': 'Carrollton', 'rivals_state': 'TX', 'rivals_hs': 'Creekview', 'rivals_pos': 'DB', 'rivals_height': 70.0, 'rivals_weight': 165.0, 'rivals_stars': 3, 'rivals_natlrk': None, 'rivals_posrk': 38, 'rivals_staterk': None}]
#Declare Field Configs
idConfigs = {'rivals' : ['playerName', 'school'], '247': ['playerName', 'school'], 'nfl' : ['player', 'college_id'], 'aa' : ['player', 'school'], 'ac' : ['playerName', 'school'], 'cr' : ['playerName', 'school']}
fieldAgg = '_'
sourceSchema = {'nfl': ['year', 'draft_round', 'draft_pick', 'team', 'pos','all_pros_first_team','pro_bowls','years_as_primary_starter','g'], 'aa' : ['year', 'afca', 'ap', 'fwaa', 'tsn', 'wcff'], 'ac' : ['year', 'coaches', 'media'], 'rivals' : ['playerName', 'city', 'state', 'highSchool', 'position', 'height', 'weight', 'stars', 'nationalRank', 'positionRank', 'stateRank'], '247' : ['playerName', 'highSchool', 'city', 'state', 'position', 'height', 'weight','nationalRank', 'positionRank', 'stateRank'], 'cr' : ['playerName', 'position', 'school', 'gamesPlayed']}

In [ ]:
##ID consists of two components
##Player Name - which is coming from a string
##School - which can be either a list or a sting.  And the list can have either 1 or 2 parts
##Final ID for everyone - list, potentially multivalued where each part equals name+_+school

def createNewID (fieldList, thisDict, fieldAgg):
    finalID= []
    for i in thisDict:
        for idx, val in enumerate(fieldList):
            if (type(i[val]) is list):
                for x in i[val]:
                    multiFinalID = []
                    cleanedName = ''.join(e for e in x if e.isalnum())
                    multiFinalID.append(cleanedName.lower().replace("jr.", "").replace("st.", "state"))
                i[val] = multiFinalID
                if (len(fieldList) -1 == idx):
                    finalID += str(i[val]).strip('[]').strip("''")
            elif (type(val) is not list):
                cleanedName = ''.join(e for e in i[val] if e.isalnum())
                i[val] = cleanedName.lower().replace("jr.", "").replace("st.", "state")
                if (len(fieldList) - 1 == idx):
                    finalID += i[val]
                else:
                    finalID = i[val] + fieldAgg
                i['ID'] = finalID
        i['ID'] = finalID
        finalID=''

In [ ]:
#Updated a bit with levensthein distance
def equals(first, second):
    # Return true if the two ints are equal.
    if (isinstance(first, str) and isinstance(second, str)):
        if (first == second):
            return True
        elif (lev.distance(first, second) < 3):
            #print (str(lev.distance(first,second)) + " " + first + " " + second)
            return True
        else:
            return False
    else:
        print(first)
        print(second)

In [ ]:
def comparator (field1, field2):
    areTheyEqual = False
    if field2 is not list:
        if equals(field1, field2):
            areTheyEqual =True
    elif field2 is list:
        for x in field2:
            if (field1 == x):
                areTheyEqual = True
    return areTheyEqual

In [ ]:
#function to join to nfl draft
def joinData (masterDict, secondDict, schema):
    for player in masterDict:
        for record in secondDict:
            if ('school' in player.keys() and 'school' in record.keys()): 
                if (player['school'] == record['school'] and (int(player['year']) == int(record['year']))):
                    if (comparator(player['ID'],record['ID'])):
                        player[schema] = {}
                        for x in sourceSchema[schema]:
                            try:
                                player[schema][x] = record[x]
                            except KeyError:
                                print(player['ID'])
                        break;


In [ ]:
createNewID(idConfigs['247'], main247, fieldAgg)
createNewID(idConfigs['rivals'], mainRivals, fieldAgg)
createNewID(idConfigs['nfl'], nfl_draft_data, fieldAgg)
createNewID(idConfigs['aa'], aaData, fieldAgg)
createNewID(idConfigs['ac'], mainAllConf, fieldAgg)
createNewID(idConfigs['cr'], mainCfbRef, fieldAgg)

In [ ]:
# This creates the root of each recruit in the JSON.  Anything that can be pulled out from the various sources is here
masterData = []
for recruit in main247:
    print(recruit)
    masterRecruit = {}
    masterRecruit['ID'] = recruit['ID']
    masterRecruit['year'] = recruit['year']
    masterRecruit['school'] = recruit['school']
    masterRecruit['compositeScore'] = recruit['compRating'].strip()
    masterRecruit['compositeStars'] = recruit['compStars']
    masterRecruit['compNationalRank'] = recruit['nationalRank']
    masterRecruit['compStateRank'] = recruit['stateRank']
    masterRecruit['compPositionRank'] = recruit['positionRank']
    masterData.append(masterRecruit)    

{'school': 'nebraska', 'year': '2012', 'playerName': 'paulthurston', 'highSchool': 'Arvada West', 'city': 'Arvada', 'state': 'CO', 'position': 'OT', 'height': 77.0, 'weight': '275', 'compRating': ' 0.9407  ', 'compStars': 4, 'nationalRank': '151', 'positionRank': '15', 'stateRank': '2', '247Rating': ' 92  ', '247Stars': 4, '247positionRank': '26', '247stateRank': '2', 'ID': 'paulthurston_nebraska'}
{'school': 'nebraska', 'year': '2012', 'playerName': 'gregmcmullen', 'highSchool': 'Archbishop Hoban', 'city': 'Akron', 'state': 'OH', 'position': 'SDE', 'height': 77.0, 'weight': '255', 'compRating': ' 0.9099  ', 'compStars': 4, 'nationalRank': '259', 'positionRank': '14', 'stateRank': '18', '247Rating': ' 95  ', '247Stars': 4, '247nationalRank': '139', '247positionRank': '6', '247stateRank': '8', 'ID': 'gregmcmullen_nebraska'}
{'school': 'nebraska', 'year': '2012', 'playerName': 'jordanwesterkamp', 'highSchool': 'Montini Catholic', 'city': 'Lombard', 'state': 'IL', 'position': 'WR', 'heigh

KeyError: 'compRating'

In [ ]:
joinData(masterData, main247, '247')
joinData(masterData,mainRivals, 'rivals')
joinData(masterData,nfl_draft_data,  'nfl')
joinData(masterData, aaData,  'aa')
joinData(masterData, mainAllConf, 'ac')
joinData(masterData, mainCfbRef, 'cr')


In [ ]:
with open("final_output_mar23.json", "w") as write_file:
    json.dump(masterData, write_file)

# now we will open a file for writing 
data_file = open('outputLinkedRecruits.csv', 'w') 
  
# create the csv writer object 
csv_writer = csv.writer(data_file) 
count= 0

for recruit in masterData: 
    if count == 0: 
  
        # Writing headers of CSV file 
        header = recruit.keys() 
        csv_writer.writerow(header) 
        count += 1
    else:
        csv_writer.writerow(recruit.values()) 

data_file.close()

In [ ]:
debugRecruits = []
debugRecruit = {}
for recruit in masterData:
    debugRecruit['ID'] = recruit['ID']
    debugRecruit['year'] = recruit['year']
    debugRecruit['school'] = recruit['school']
    if ('compRating' in recruit.keys()):
        debugRecruit['247'] = recruit['compRating']
    else:
        debugRecruit['247'] = 'None'
    #print(recruit)
    if ('rivals' in recruit.keys()):
        debugRecruit['rivals'] = 'true'
    else:
        debugRecruit['rivals'] = 'false'
    debugRecruits.append(debugRecruit)
    debugRecruit= {}

#print(debugRecruits)
    
with open("debug_output_mar7.json", "w") as write_file:
    json.dump(debugRecruits, write_file)

In [ ]:
# now we will open a file for writing 
data_file = open('debugRecruit.csv', 'w') 
  
# create the csv writer object 
csv_writer = csv.writer(data_file) 
count= 0

for recruit in debugRecruits: 
    if count == 0: 
  
        # Writing headers of CSV file 
        header = recruit.keys() 
        csv_writer.writerow(header) 
        count += 1
  
    # Writing data of CSV file 
    if (recruit['rivals'] == 'false'):
        csv_writer.writerow(recruit.values()) 

data_file.close()